# Traveling Salesman Problem with Gmaps
In this session, we will solve a fictitious case of TSP using pulp for optimization and gmaps for visualization and routing. I'm using the location data of a retail company as an illustration of the implementation. Product X will be sent from Alfamidi (Depot) to six Alfamarts. It is assumed that the truck has unlimited capacity so that it can directly meet the demands of the six retailers in one go.

## 1. Importing Library and Data

In [1]:
import numpy as np
from pulp import *
import gmaps
import googlemaps
import pandas as pd

In [2]:
#Input the API key to create a map and access the distance between two places
#If you don't have the key, please register at https://developers.google.com/maps

API_KEY = 'Input your API Key here!'
gmaps.configure(api_key=API_KEY)
googlemaps = googlemaps.Client(key=API_KEY)

In [3]:
#I get the data manually using gmaps

data = {"Name":["Depot",
                "Alfamart Bugisan",
                "Alfamart Ring Road Selatan",
                "Alfamart Bibis",
                "Alfamart UMY Tamantirto",
                "Alfamart Ngebel",
                "Alfamart Ring Road Selatan 2"],
        "Location":[(-7.820275519559818, 110.35576478748318),
                    (-7.818459939237757, 110.34828905480705),
                    (-7.826882740143658, 110.3457904293808),
                    (-7.825849387004239, 110.32770847947735),
                    (-7.814562872952285, 110.32839097560857),
                    (-7.814073787921593, 110.3183264772633),
                    (-7.809770036857429, 110.32472496772033)]}

alfa = pd.DataFrame(data)
alfa

,Name,Location
0,Depot,"(-7.820275519559818, 110.35576478748318)"
1,Alfamart Bugisan,"(-7.818459939237757, 110.34828905480705)"
2,Alfamart Ring Road Selatan,"(-7.826882740143658, 110.3457904293808)"
3,Alfamart Bibis,"(-7.825849387004239, 110.32770847947735)"
4,Alfamart UMY Tamantirto,"(-7.814562872952285, 110.32839097560857)"
5,Alfamart Ngebel,"(-7.814073787921593, 110.3183264772633)"
6,Alfamart Ring Road Selatan 2,"(-7.809770036857429, 110.32472496772033)"


## 2. Map Visualization

In [4]:
#See the location

alfa_map = gmaps.figure()
depot = gmaps.symbol_layer([alfa.Location[0]],fill_color="blue",stroke_opacity=0,
                           scale=6,info_box_content="Depot",display_info_box=True)

markers = gmaps.marker_layer(alfa.Location[1:])
 
alfa_map.add_layer(depot)    
alfa_map.add_layer(markers)
    
alfa_map

Figure(layout=FigureLayout(height='420px'))

The output display will be like the image below:

<img src="https://user-images.githubusercontent.com/61647791/144628288-870c3c6a-ca83-4d02-a273-48b6cd4866be.png" />

## 3. Distance Between Two Places
The next step is to calculate the distance between two places using the direction API from gmaps. The distance used is the closest distance between two places with "driving" mode. The result then entered into a square matrix with the length and width according to the number of locations including the depot. Here is the code:

In [5]:
#Function to create distance matrix
def distance_matrix(loc_column):
    
    distance_result = np.zeros((len(loc_column),len(loc_column)))
    
    for i in range(len(loc_column)):
        for j in range(len(loc_column)):
            
            # menghitung jarak antar lokasi
            api_result = googlemaps.directions(loc_column[i],
                                               loc_column[j],
                                               mode = 'driving')
            
            # memasukkan hasil perhitungan pada matriks 
            distance_result[i][j] = api_result[0]['legs'][0]['distance']['value']
    
    return distance_result

In [6]:
#We can convert the distance matrix into Pandas DataFrame
#Distance in meter unit

distance_data = pd.DataFrame(distance_matrix(alfa.Location),columns=alfa.Name,index=alfa.Name)
distance_data

Name,Depot,Alfamart Bugisan,Alfamart Ring Road Selatan,Alfamart Bibis,Alfamart UMY Tamantirto,Alfamart Ngebel,Alfamart Ring Road Selatan 2
Name,,,,,,,
Depot,0.0,1349.0,1782.0,4264.0,4693.0,5764.0,5823.0
Alfamart Bugisan,1349.0,0.0,2479.0,3536.0,3002.0,5036.0,4132.0
Alfamart Ring Road Selatan,2518.0,1217.0,0.0,2482.0,2911.0,3982.0,4041.0
Alfamart Bibis,4269.0,3552.0,3155.0,0.0,1279.0,2350.0,2409.0
Alfamart UMY Tamantirto,4667.0,3002.0,4176.0,1280.0,0.0,1689.0,1130.0
Alfamart Ngebel,6246.0,5529.0,5755.0,2827.0,1695.0,0.0,3291.0
Alfamart Ring Road Selatan 2,5633.0,4916.0,5142.0,2245.0,1070.0,1732.0,0.0


In [16]:
distance_data.to_csv("alfa_distance.csv")

## 4. Model Creation
The TSP model that I made refers to the Miller Tucker Zemlin Subtour Elimination Constraint. The MTZ method uses a dummy variable u which is an integer type as many as the number of locations. The following is the formulation of the mathematical model.


<img src="https://user-images.githubusercontent.com/61647791/145667573-570d0e2e-665a-4c7e-971c-e4bed321e31d.png" height=450, width=450/>

In [7]:
model = LpProblem("TSP",LpMinimize)

#decision variable
keys = [(i,j) for i in alfa.Name for j in alfa.Name]
x = LpVariable.dicts("x",keys,cat="Binary")

#dummy variable for MTZ
u = LpVariable.dicts("u",(i for i in alfa.Name), lowBound=1, upBound=len(alfa.Name))

In [8]:
#Objective Function
model += lpSum(distance_data.loc[i,j]*x[i,j] for (i,j) in keys)

In [9]:
#make sure the truck doesn't go around in the same location, for example from depot to depot
#make sure the truck from location i go to one of the unvisited locations 
#make sure location j can only be visited by trucks from one location

for i in alfa.Name:
    model += x[i,i]==0
    model += lpSum(x[i,j] for j in alfa.Name) == 1
    
for j in alfa.Name:
    model += x[j,j] == 0
    model += lpSum(x[i,j] for i in alfa.Name) == 1

#MTZ constrain
#make sure i is not equal to j and does not include depot
for i in alfa.Name:
    for j in alfa.Name:
        if i!=j and (i!="Depot" and j!="Depot"):
            model += u[j]>=u[i]+1 -(len(alfa.Name))*(1-x[i,j])

In [10]:
#Function to find a subtour and routing result

def get_plan(r0):
    r=copy.copy(r0)
    route = []
    while len(r) != 0:
        plan = [r[0]]
        del (r[0])
        l = 0
        while len(plan) > l:
            l = len(plan)
            for i, j in enumerate(r):
                if plan[-1][1] == j[0]:
                    plan.append(j)
                    del (r[i])
        route.append(plan)
    return(route)

In [11]:
import copy

status=model.solve()

print("-----------------")
print(status,LpStatus[status],value(model.objective))
route=[(i,j) for i,j in keys if value(x[i,j])==1]

-----------------
1 Optimal 14451.0


In [12]:
print(get_plan(route))

[[('Depot', 'Alfamart Ring Road Selatan'), ('Alfamart Ring Road Selatan', 'Alfamart Bibis'), ('Alfamart Bibis', 'Alfamart Ring Road Selatan 2'), ('Alfamart Ring Road Selatan 2', 'Alfamart Ngebel'), ('Alfamart Ngebel', 'Alfamart UMY Tamantirto'), ('Alfamart UMY Tamantirto', 'Alfamart Bugisan'), ('Alfamart Bugisan', 'Depot')]]


## 4. Route Visualization

In [15]:
# visualization : plotting on google maps
alfa_index = alfa.set_index("Name")

fig = gmaps.figure()
layer = []

color_list = ["#b0bf1a","#800869","#6897bb","#921515","#32f0ff","#2aa63d","#0e2f44"]

#Draw a route
for r,c in zip(route,color_list):
    layer.append(gmaps.directions.Directions(alfa_index.loc[r[0]]["Location"],
                                             alfa_index.loc[r[1]]["Location"],
                                             mode="car",stroke_weight=5, stroke_color=c,
                                             show_markers=False))

for i in range(len(layer)):
    fig.add_layer(layer[i])    
    
#Add a marker
route_plan = {n[0]:i for n,i in zip(get_plan(route)[0],range(0,len(route)))}
alfa_index["Destination"] = alfa_index.index.map(route_plan)

dict_plan = [{"Name":n,"Destination":i} for n,i in zip(alfa_index.index, alfa_index.Destination)]
info_box_template = """
                        <dl>
                        <dt>Location</dt><dd>{Name}</dd>                    
                        <dt>Destination number</dt><dd>{Destination}</dd>
                        </dl>
                    """
info = [info_box_template.format(**r)for r in dict_plan]


markers = gmaps.marker_layer(alfa_index.Location, info_box_content=info)
        


fig.add_layer(markers)
    
fig

Figure(layout=FigureLayout(height='420px'))

The output display will be like the image below:

<img src="https://user-images.githubusercontent.com/61647791/147240808-1dcf6839-abe7-46c6-9ba1-3da4443e5664.png" />

In [117]:
#If you want to save map
from ipywidgets.embed import embed_minimal_html

embed_minimal_html('alfa.html', views=[fig])

## Conclusion

1. Based on the optimization results, the total minimum distance traveled to distribute products to six depots is 14,451 km.
2. The total distance is quite large due to road conditions in the form of a *ring road* so that trucks need to rotate to change locations.
3. The order of the route chosen is Depot -> South Alfamart Ring Road -> Alfamart Bibis -> South Alfamart Ring Road 2 -> Alfamart Ngebel -> Alfamart UMY Tamantirto -> Alfamart Bugisan.
4. Even though from the map, Alfamart Bibis and Alfamart UMY Tamantirto look quite close, the two are separated by a ring road so that Alfamart UMY Tamantirto becomes the second last destination before heading to Alfamart Bugisan.
5. This TSP model assumes that trucks can meet all retailer's demands in one go without going back and forth. Whereas in real conditions, trucks have a limited capacity. In addition, the demands of one retailer may differ from one another. The Vehicle Routing Problem model is the solution to this problem.
